In [3]:
import time
import datetime
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support import expected_conditions as EC

from concurrent.futures import ThreadPoolExecutor
import multiprocessing

from Functions.function import movies_for_category, categories, print_process, movies, inicio_driver, series_informacion
from bs4 import BeautifulSoup
import requests
import re

import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException


In [4]:
print_process("Comienzo del Script")

|-----------------------------Comienzo del Script-----------------------------|


In [5]:
categorias = categories()

|---------------------Categorias capturadas correctamente.---------------------|
|-----------------------Categorias demoro 9.84 segundos-----------------------|


In [6]:

# Obtener la cantidad de núcleos disponibles en el sistema
num_nucleos = multiprocessing.cpu_count()

# Crear un diccionario para almacenar los resultados
resultados_dict = {}

categorias_comienzo = time.time()

with ThreadPoolExecutor(max_workers=num_nucleos) as executor:
    # Ejecutar la función para cada categoría en paralelo
    resultados = list(executor.map(movies_for_category, categorias))
    
    # Asignar los resultados al diccionario
    for categoria, resultado in zip(categorias, resultados):
        resultados_dict[categoria] = resultado

peliculas_total = []

for x,i in resultados_dict.items():
    for e in i:
        peliculas_total.append(e)

peliculas_total = set(peliculas_total)

categorias_fin = time.time()

print_process(f"La extraccion total de las categorias tubo una demora de {round((categorias_fin-categorias_comienzo),2)} segundos")


|Todo el contenido de la categoria 'biography' fue capturado correctamente, demoro 25.49 segundos|
|Todo el contenido de la categoria 'crime' fue capturado correctamente, demoro 26.13 segundos|
|Todo el contenido de la categoria 'history' fue capturado correctamente, demoro 26.15 segundos|
|Todo el contenido de la categoria 'fantasy' fue capturado correctamente, demoro 27.02 segundos|
|Todo el contenido de la categoria 'family' fue capturado correctamente, demoro 27.53 segundos|
|Todo el contenido de la categoria 'adventure' fue capturado correctamente, demoro 27.64 segundos|
|Todo el contenido de la categoria 'animation' fue capturado correctamente, demoro 29.44 segundos|
|Todo el contenido de la categoria 'action' fue capturado correctamente, demoro 31.78 segundos|
|Todo el contenido de la categoria 'horror' fue capturado correctamente, demoro 41.51 segundos|
|Todo el contenido de la categoria 'musical' fue capturado correctamente, demoro 10.56 segundos|
|Todo el contenido de la cate

In [7]:
peliculas_total = [x for x in peliculas_total if "/content/" in x]

In [ ]:
print_process("Comienzo del proceso de extraccion de informacion de cada iteracion")

In [8]:
# Crear un diccionario para almacenar los resultados
todo = {"titulo": [],"idioma" : [], "subtitulos" : [], "ano": [], "duracion_minutos": [], "categorias": [], "sinopsis": [], "tipo": [], "link": []}

comienzo_movies = time.time()

with ThreadPoolExecutor(max_workers=num_nucleos) as executor:
    # Ejecutar la función para cada categoría en paralelo
    resultados = list(executor.map(movies, peliculas_total))

    try:
        for resultado in resultados:
            for clave, valor in resultado.items():
                todo[clave].append(valor)
    except:
        print()

fin_movies = time.time()

print_process(f"El proceso de captura de la informacion de todo el contenido fue exsitoso, y demoro {round((fin_movies - comienzo_movies),2)} segundos")


Error al cargar la url https://filmzie.com/content/the-devil-at-your-heels en el intento 1: 404
|El proceso de captura de la informacion de todo el contenido fue exsitoso, y demoro 360.36 segundos|


In [9]:
df = pd.DataFrame(todo)

In [10]:

df.to_csv("Bases_datos/Peliculas_Series_Completo_Respaldo.csv" , index=False)

In [11]:
peliculas = df.loc[df["tipo"] == "Pelicula"].reset_index(drop=True)
series = df.loc[df["tipo"] == "Serie"].reset_index(drop=True)

In [12]:

peliculas["duracion_minutos"] = peliculas["duracion_minutos"].str.replace(" min","").astype(int)


In [13]:
peliculas

,titulo,idioma,subtitulos,ano,duracion_minutos,categorias,sinopsis,tipo,link
0,Cherry,English,English,2010,99,"Comedy, Drama",An Ivy League freshman discovers unexpected le...,Pelicula,https://filmzie.com/content/cherry-2010
1,The Blackwell Ghost 1,English,English,2017,59,"Horror, Mystery",Embark on a thrilling journey as a journalist ...,Pelicula,https://filmzie.com/content/the-blackwell-ghos...
2,3 Cents a Kilo,Chinese,English,2021,23,"Documentary, Short",3 Cents A Kilo documents the day-to-day lives ...,Pelicula,https://filmzie.com/content/3-cents-a-kilo-2021
3,A Living Dog,English,"German, English, Spanish, Chinese",2019,94,"Action, Sci-Fi",Join a wounded deserter and a determined femal...,Pelicula,https://filmzie.com/content/a-living-dog-2019
4,Jango on Tour,Swedish,"German, English",2011,78,Comedy,"A worthless movie director, the singer-songwri...",Pelicula,https://filmzie.com/content/jango-on-tour-2011
...,...,...,...,...,...,...,...,...,...
2199,Mike Case in: The Big Kiss Off,English,English,2013,74,Comedy,"Mike Case, a car-dwelling detective, reluctant...",Pelicula,https://filmzie.com/content/mike-case-in-the-b...
2200,Prime Suspects,English,English,1997,91,"Action, Thriller","In a film noir feast for the millennium, a col...",Pelicula,https://filmzie.com/content/prime-suspects-1997
2201,Property,English,English,2020,27,"Short, Drama, Thriller",A devoted woman fights to free an enslaved gir...,Pelicula,https://filmzie.com/content/property-2020
2202,Killing Anabella,English,English,2018,13,"Animation, Short","Meet Anabella, a complex character haunted by ...",Pelicula,https://filmzie.com/content/killing-anabella


In [14]:
peliculas.to_csv("Bases_datos/Peliculas.csv",index=False)

In [15]:
series.drop(columns=['duracion_minutos'], inplace=True)

series

,titulo,idioma,subtitulos,ano,categorias,sinopsis,tipo,link
0,Day Zero,English,Subtitles,2020,Documentary,Day Zero can be a starting point or an end. We...,Serie,https://filmzie.com/content/day-zero-2020
1,New Space Adventures,English,Subtitles,2020,Documentary,New Space Adventures will closely explore seve...,Serie,https://filmzie.com/content/new-space-adventur...
2,Welcome Home,English,Subtitles,2015,"Comedy, Family",When three thirty-somethings have no choice bu...,Serie,https://filmzie.com/content/welcome-home-2015
3,Animal Armory,English,Subtitles,2016,"Documentary, European Production",The lives of animals are not peaceful. They mu...,Serie,https://filmzie.com/content/animal-armory-2016
4,Gugu,English,Subtitles,2015,"Animation, Family",Baby elephant Gugu gets captured by a band of ...,Serie,https://filmzie.com/content/gugu-2015
...,...,...,...,...,...,...,...,...
94,Korban,Malay (macrolanguage),Subtitles,2021,Drama,"AISYAH, a sous-chef under renowned CHEF LOUIS,...",Serie,https://filmzie.com/content/korban-2021
95,Rescue Warriors,English,Subtitles,2005,Documentary,Discover the rigorous journey of USAF Pararesc...,Serie,https://filmzie.com/content/rescue-warriors-2005
96,Chemistry Series,English,Subtitles,1983,Documentary,Explore the world of chemistry through these i...,Serie,https://filmzie.com/content/chemistry-series-1983
97,Close Quarter Battle,English,Subtitles,2012,"Documentary, European Production",U.S. Special Forces veteran Terry Schappert ho...,Serie,https://filmzie.com/content/close-quarter-batt...


In [16]:
links_series = list(series["link"].values)

links_series

['https://filmzie.com/content/day-zero-2020',
 'https://filmzie.com/content/new-space-adventures-2020',
 'https://filmzie.com/content/welcome-home-2015',
 'https://filmzie.com/content/animal-armory-2016',
 'https://filmzie.com/content/gugu-2015',
 'https://filmzie.com/content/wonder-world-of-science-1989',
 'https://filmzie.com/content/heroes-manufactured-creators-unleashed-2020',
 'https://filmzie.com/content/the-benefits-of-gusbandry-2016',
 'https://filmzie.com/content/2nd-generation-2018',
 'https://filmzie.com/content/exotic-destinations-101-2019',
 'https://filmzie.com/content/the-shadow-dimension-2021',
 'https://filmzie.com/content/bill-tilghman-and-the-outlaws-2019',
 'https://filmzie.com/content/wilderness-to-table-with-chef-bri-2021',
 'https://filmzie.com/content/the-divine-mediator-2018',
 'https://filmzie.com/content/the-beech-boys-2019',
 'https://filmzie.com/content/quaranteam-2020',
 'https://filmzie.com/content/alaska-outdoors-magazine-2023',
 'https://filmzie.com/con

In [17]:
testeo = links_series

# Obtener la cantidad de núcleos disponibles en el sistema
num_nucleos = multiprocessing.cpu_count()

# Crear un diccionario para almacenar los resultados
resultados_dict = {"titulo": [], "temporadas": [], "capitulos": []}

categorias_comienzo = time.time()

with ThreadPoolExecutor(max_workers=num_nucleos) as executor:
    # Ejecutar la función para cada categoría en paralelo
    resultados_temporales = list(executor.map(series_informacion, testeo))

# Separar los resultados temporales en listas individuales
titulos, temporadas, capitulos = zip(*resultados_temporales)

# Agregar los resultados a los diccionarios
resultados_dict["titulo"].extend(titulos)
resultados_dict["temporadas"].extend(temporadas)
resultados_dict["capitulos"].extend(capitulos)

categorias_fin = time.time()

print_process(f"La extraccion total de las categorias tuvo una demora de {round((categorias_fin-categorias_comienzo), 2)} segundos")

Tiempo total: 21.418156147003174 segundos
Tiempo total: 21.638126611709595 segundos
Tiempo total: 23.17629098892212 segundos
Tiempo total: 23.272035121917725 segundos
Tiempo total: 23.305944442749023 segundos
Tiempo total: 23.416161060333252 segundos
Tiempo total: 23.461038827896118 segundos
Tiempo total: 23.48298168182373 segundos
Tiempo total: 23.67198872566223 segundos
Tiempo total: 23.948248624801636 segundos
Tiempo total: 25.537598133087158 segundos
Tiempo total: 25.94104266166687 segundos
Tiempo total: 24.053098678588867 segundos
Tiempo total: 25.873071432113647 segundos
Tiempo total: 23.88803005218506 segundos
Tiempo total: 24.2106773853302 segundos
Tiempo total: 24.446067810058594 segundos
Tiempo total: 24.66350555419922 segundos
Tiempo total: 26.856983423233032 segundos
Tiempo total: 24.990105628967285 segundos
Tiempo total: 25.980518579483032 segundos
Tiempo total: 27.138954639434814 segundos
Tiempo total: 26.355393648147583 segundos
Tiempo total: 18.996880531311035 segundos


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/main/div/div/div[2]/div/div[2]/h1"}
  (Session info: chrome=124.0.6367.202); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0025C113+48259]
	(No symbol) [0x001ECA41]
	(No symbol) [0x000E0A17]
	(No symbol) [0x00120BED]
	(No symbol) [0x00120C9B]
	(No symbol) [0x0015BC12]
	(No symbol) [0x00140DE4]
	(No symbol) [0x00159B9C]
	(No symbol) [0x00140B36]
	(No symbol) [0x0011570D]
	(No symbol) [0x001162CD]
	GetHandleVerifier [0x005165A3+2908435]
	GetHandleVerifier [0x00553BBB+3159851]
	GetHandleVerifier [0x002F50CB+674875]
	GetHandleVerifier [0x002FB28C+699900]
	(No symbol) [0x001F6244]
	(No symbol) [0x001F2298]
	(No symbol) [0x001F242C]
	(No symbol) [0x001E4BB0]
	BaseThreadInitThunk [0x75D8FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77667C5E+286]


In [18]:
df_1 = pd.DataFrame(resultados_dict)
df_1

,titulo,temporadas,capitulos
